Remove Noisy images

In [ ]:
img_exts = ['jpg', 'jpeg', 'png', 'bmp']

In [ ]:

import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
import cv2
import imghdr
from tensorflow.keras.preprocessing import image_dataset_from_directory


data_dir = '../Preprocessing/Processed_Images'

for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in img_exts:
                print(image_path)    # print the path of the image with unknown extension
                os.remove(image_path)
        except Exception as e:
            print("Issue with image:" .format(image_path))
            
            


Load Dataset

In [ ]:
batch_size = [16,32]
for size in batch_size:
   data = tf.keras.utils.image_dataset_from_directory('../Preprocessing/Processed_Images', batch_size=size, image_size=(256, 256), shuffle=True) #Data pipeline

class_names = data.class_names #get the class names
print(class_names)

data_iterator = data.as_numpy_iterator() #allows us to access Data pipeline

batch = data_iterator.next() #get the next batch of data



In [ ]:
batch[0].shape # shape of the batch

In [ ]:
batch[1] # labels

Plotting Images

In [ ]:
from matplotlib import pyplot as plt
rows = 4
cols = 8
fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(20, 12))
fig.suptitle('Sample Images with Class Labels', fontsize=16)

# Flatten axes for easier indexing
axes = axes.flatten()

# Display images in grid
for idx, (image, label) in enumerate(zip(batch[0][:32], batch[1][:32])):
    # Get class name from label index
    class_name = class_names[label]
    
    # Display image
    axes[idx].imshow(image.astype("uint8"))
    axes[idx].set_title(f'{class_name}', fontsize=8)
    axes[idx].axis('off')

# Add color-coded legend
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                             markerfacecolor=f'C{i}', markersize=10, 
                             label=name) for i, name in enumerate(class_names)]
fig.legend(handles=legend_elements, loc='center right')

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(right=0.85)  # Make room for legend
plt.show()

Preprocessing 

Scaling Data

In [ ]:
data = data.map(lambda x,y : ((x/255),y)) #normalizing the data  
scaled_iterator = data.as_numpy_iterator() #allows us to access Data pipeline
batch = scaled_iterator.next() #get the next batch of data

batch[0].max() #max value in the batch

Split data

In [ ]:
len(data)

In [ ]:
train_size = int(len(data) * 0.7) #70% of the data for training
val_size = int(len(data) * 0.2) #20% of the data for validation
test_size = int(len(data) * 0.1) #10% of the data for testing

In [ ]:
print(train_size, val_size, test_size)

In [ ]:
train = data.take(train_size) #take the first 70% of the data for training
val = data.skip(train_size).take(val_size) #skip the first 70% and take the next 20% for validation
test = data.skip(train_size + val_size).take(test_size) #skip the first 90% and take the next 10% for testing

print(len(train), len(val), len(test))

Building Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomTranslation, RandomContrast, RandomCrop
# filter size = 3x3
# input shape = 256x256x3
# stride = 1
IMAGE_SIZE = 256
CHANNELS = 3
BATCH_SIZE = batch[0].shape[0]
model = Sequential() #initialize the model



data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal_and_vertical"),         # Flip images both horizontally and vertically
    RandomRotation(0.4),                           # Rotate images up to 40% in both directions
    RandomZoom(height_factor=(-0.2, 0.2),          # Random zoom in/out
               width_factor=(-0.2, 0.2)),
    RandomTranslation(height_factor=0.2,           # Translate images up to 20% in height
                      width_factor=0.2),           # Translate images up to 20% in width
    RandomContrast(0.2),                           # Adjust contrast randomly
    RandomCrop(IMAGE_SIZE - 20, IMAGE_SIZE - 20),  # Crop random parts of the image
    tf.keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE)  # Resize back to target size
])


In [ ]:
# Checking what is the expected dimension order for channel
from tensorflow.keras import backend as k
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
batch_input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
chanDim = -1
if k.image_data_format() == "channels_first":
    input_shape = (CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
    batch_input_shape = (BATCH_SIZE, CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
    chanDim = 1

Add Layers

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Train

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
model_dir = '../saved_models'
log_dir = os.path.join(model_dir, 'logs')
os.makedirs(model_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

# Define callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath=os.path.join(model_dir, 'best_model.keras'),
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1
    ),
    TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        write_graph=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

# Train model
history = model.fit(
    train,
    validation_data=val,
    epochs=50,
    callbacks=callbacks,
    verbose=1
)

# Save final model
final_model_path = os.path.join(model_dir, 'final_model.keras')
model.save(final_model_path)
print(f"Model saved to {final_model_path}")
from matplotlib import pyplot as plt
rows = 4
cols = 8
fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(20, 12))
fig.suptitle('Sample Images with Class Labels', fontsize=16)

# Flatten axes for easier indexing
axes = axes.flatten()

# Display images in grid
for idx, (image, label) in enumerate(zip(batch[0][:32], batch[1][:32])):
    # Get class name from label index
    class_name = class_names[label]
    
    # Display image
    axes[idx].imshow(image.astype("uint8"))
    axes[idx].set_title(f'{class_name}', fontsize=8)
    axes[idx].axis('off')

# Add color-coded legend
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                             markerfacecolor=f'C{i}', markersize=10, 
                             label=name) for i, name in enumerate(class_names)]
fig.legend(handles=legend_elements, loc='center right')

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(right=0.85)  # Make room for legend
plt.show()

print(train_size, val_size, test_size)

train = data.take(train_size) #take the first 70% of the data for training
val = data.skip(train_size).take(val_size) #skip the first 70% and take the next 20% for validation
test = data.skip(train_size + val_size).take(test_size) #skip the first 90% and take the next 10% for testing

print(len(train), len(val), len(test))

In [ ]:
history.history

Overall Code

In [ ]:

import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
import cv2
import imghdr
from tensorflow.keras.preprocessing import image_dataset_from_directory

img_exts = ['jpg', 'jpeg', 'png', 'bmp']
data_dir = 'skinType'

for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in img_exts:
                print(image_path)    # print the path of the image with unknown extension
                os.remove(image_path)
        except Exception as e:
            print("Issue with image:" .format(image_path))
batch_size = [16,32]
for size in batch_size:
   data = tf.keras.utils.image_dataset_from_directory('skinType', batch_size=size, image_size=(256, 256), shuffle=True) #Data pipeline

class_names = data.class_names #get the class names
print(class_names)

data_iterator = data.as_numpy_iterator() #allows us to access Data pipeline

batch = data_iterator.next() #get the next batch of data

batch[0].shape # shape of the batch
       
batch[1] # labels

data = data.map(lambda x,y : ((x/255),y)) #normalizing the data  
scaled_iterator = data.as_numpy_iterator() #allows us to access Data pipeline
batch = scaled_iterator.next() #get the next batch of data

batch[0].max() #max value in the batch

len(data)

train_size = int(len(data) * 0.7) #70% of the data for training
val_size = int(len(data) * 0.2) #20% of the data for validation
test_size = int(len(data) * 0.1) #10% of the data for testing


New Code

In [ ]:
import os
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from vit_pytorch import ViT
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader
from pytorch_lamb import Lamb
from torch.optim import AdamW
import timm 
from sklearn.metrics import classification_report, confusion_matrix
#unprocessed Data
# dataset_path = '../skinType' 

#preprocessed Data
dataset_path = '../Preprocessing/Processed_Images'
#dataset_path="../skintypepatches 128x128"
#_FaceCrops_256' 

# Define hyperparameters
batch_sizes = [ 8, 32,64]
learning_rates =[0.01, 0.001]
optimizers_list = [ 'LAMB' ,'AdamW']

num_classes = 3  # Dry, Normal, Oily skin types
#class_names = ['oily', 'dry', 'normal']
steps_per_epoch = 20 
total_epochs=500

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# Load Dataset
transform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])
# train_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(10),
#     transforms.ToTensor(),
#     transforms.Normalize(mean, std)
# ])

# val_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean, std)
# ])
# Dataset Path
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)


train_size = int(0.7 * len(dataset))
val_size = int(0.20 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

#val_dataset.dataset.transform = val_transform
print(f"train size {train_size}")
print(f"Val size {val_size}")
print(f"Test Size {test_size}")

# print(f"train Dataset {train_dataset}")
# print(f"Val Dataset {val_dataset}")
# print(f"Test Dataset {test_dataset}")

# Function to initialize the model
# def create_vit_model():    
#     model = ViT(
#         image_size=128,
#         patch_size=16,
#         num_classes=num_classes,  
#         dim=512,                      
#         heads=8,  
#         depth=8,              
#         mlp_dim=512,          
#         dropout=0.4,           
#         emb_dropout=0.4        
#     )
    
#     return model

# Optimizer choices
def get_optimizer(optimizer_name, model_params, lr,weight_decay=0.001):
    if optimizer_name == 'AdamW':
        return optim.AdamW(model_params, lr=lr,weight_decay=weight_decay)
    elif optimizer_name=='LAMB':
        return Lamb(model_params, lr=lr,weight_decay=weight_decay)

# Training Function
def train_model(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []
    y_true, y_pred = [], []
    all_probs = []

    for inputs, labels in tqdm(loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.detach().cpu().numpy())
        
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

    try:
        unique_classes = np.unique(all_labels)
        if len(unique_classes) == num_classes:
            roc_auc = roc_auc_score(all_labels, np.array(all_probs), multi_class='ovr', average='weighted')
        else:
            roc_auc = None
    except ValueError:
        roc_auc = None 

    rmse = np.sqrt(mean_squared_error(all_labels, all_preds))

    return running_loss / len(loader), accuracy, precision, recall, f1, roc_auc, rmse

# Validation and Test Function
def evaluate_model(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []
    y_true, y_pred = [], []
    
    all_probs = [] 

    with torch.no_grad():
        for inputs, labels in tqdm(loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            
            loss = criterion(outputs, labels)
            running_loss += loss.item()


            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    
    try:
       
        unique_classes = np.unique(all_labels)
        if len(unique_classes) == num_classes:
            roc_auc = roc_auc_score(all_labels, np.array(all_probs), multi_class='ovr', average='weighted')
        else:
            print("Warning: Not all classes are present in the evaluation set. Skipping ROC-AUC calculation.")
            roc_auc = None
    except ValueError:
        roc_auc = None  

    rmse = np.sqrt(mean_squared_error(all_labels, all_preds))
    
    return running_loss / len(loader), accuracy, precision, recall, f1, roc_auc, rmse


# Main training loop
def train_and_evaluate(batch_size, lr, optimizer_name):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)

    # Dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    # Model, Optimizer, Criterion
    # model = create_vit_model().to(device)
    model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
    model.to(device)
    optimizer = get_optimizer(optimizer_name, model.parameters(), lr)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    # Training
    for epoch in range(100,total_epochs,20):
        # print("\n\n-------------------------Checking Weights in Each Iteration----------------------------")
        # print("Initial Weights:")
        # print_weights(model)
        # print("\n\n-------------------------Checking Weights in Each Iteration-----------------------------")
  
        train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
        
        val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse= evaluate_model(model, val_loader, criterion, device)
        test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)
        
        scheduler.step()
        scheduler.step(val_loss)
        print("----------Values After Training-----------")
        print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
              f"\nOptimizer: {optimizer_name} \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                   f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
                   f"Train ROC AUC: {train_roc_auc:.4f}, Train RMSE: {train_rmse:.4f}")
        
        print("\n\n-----------Values After Validation-----------")
        print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
              f"\nOptimizer: {optimizer_name} \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                    f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc:.4f}, "
                    f"Val RMSE: {val_rmse:.4f}")
        
        print("\n\n-----------Values After Testing-----------")
        print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
              f"\nOptimizer: {optimizer_name} Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                    f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}, Test RMSE: {test_rmse:.4f}")
                
        
       
        
        #Saving Training data
        overall_result = {
            'Epoch': epoch + 1,
            'Batch Size': batch_size,
            'Learning Rate': lr,
            'Optimizer': optimizer_name,
            
            'Train Loss':round(train_loss, 4),
            'Test Loss':round(test_loss, 4),
            'Val Loss':round(val_loss, 4),
            
            'Train Acc': round(train_acc, 4),
            'Test Acc': round(test_acc, 4),
            'Val Acc': round(val_acc, 4),
            
            
            'Train Precision': round(train_precision, 4),
            'Test Precision': round(test_precision, 4),
            'Val Precision': round(val_precision, 4),
            
            'Train Recall': round(train_recall, 4),
            'Test Recall': round(test_recall, 4),
            'Val Recall': round(val_recall, 4),
            
            'Train F1 Score': round(train_f1, 4),
            'Test F1 Score': round(test_f1, 4),
            'Val F1 Score': round(val_f1, 4),
            
            'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
            'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
            'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,
            
            'Train RMSE': round(train_rmse, 4),
            'Test RMSE': round(test_rmse, 4),
            'Val RMSE': round(val_rmse, 4)
        }
        
        # Append to CSV
        overall_result_file = 'pretrained_overall_result.csv'
        
        if not os.path.isfile(overall_result_file):
            pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)        
        else:
            pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)
            
    # Save the trained model
    os.makedirs('../saved_models', exist_ok=True)
    model_save_path = f"../saved_models/vit_model_bs{batch_size}_lr{lr}_optimizer{optimizer_name}.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")

    # # Convert results into DataFrame
    df_results = pd.read_csv('pretrained_overall_result.csv')

    # Reshape the dataframe to long format for seaborn
    df_long = pd.melt(df_results, id_vars=['Epoch'], var_name='Metric', value_name='Value')

    # Optional: Drop rows with NaN values in the Value column (if any)
    df_long = df_long.dropna(subset=['Value'])

    # Set up the plots with a style
    sns.set_theme(style="whitegrid")

    # Plot Loss (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train Loss', 'Test Loss', 'Val Loss'])],x='Epoch', y='Value', hue='Metric')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(title='Metric')
    plt.show()

    # Plot Accuracy (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train Acc', 'Test Acc', 'Val Acc'])],
                x='Epoch', y='Value', hue='Metric')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(title='Metric')
    plt.show()

    # Plot Precision (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train Precision', 'Test Precision', 'Val Precision'])],x='Epoch', y='Value', 
                 hue='Metric')
    plt.title('Precision Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Precision')
    plt.legend(title='Metric')
    plt.show()

    # Plot Recall (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train Recall', 'Test Recall', 'Val Recall'])],x='Epoch', y='Value', hue='Metric')
    plt.title('Recall Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.legend(title='Metric')
    plt.show()

    # Plot F1 Score (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train F1 Score', 'Test F1 Score', 'Val F1 Score'])],
                x='Epoch', y='Value', hue='Metric')
    plt.title('F1 Score Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend(title='Metric')
    plt.show()

    # Plot ROC AUC (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train ROC AUC', 'Test ROC AUC', 'Val ROC AUC'])],
                x='Epoch', y='Value', hue='Metric')
    plt.title('ROC AUC Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('ROC AUC')
    plt.legend(title='Metric')
    plt.show()

    # Plot RMSE (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train RMSE', 'Test RMSE', 'Val RMSE'])],
                x='Epoch', y='Value', hue='Metric')
    plt.title('RMSE Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('RMSE')
    plt.legend(title='Metric')
    plt.show()

    # return test_acc, precision, recall, f1, roc_auc, rmse

for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
                #test_metrics = 
                train_and_evaluate(batch_size, lr, optimizer_name)
                # print(f"Final Test Metrics with Batch Size {batch_size}, LR {lr}, Optimizer {optimizer_name}: {test_metrics}")
                
            

New Model With Clipping

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from torch_optimizer import Lamb
import clip

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Hyperparameters
batch_sizes = [8]
learning_rates = [0.01, 0.001]
optimizers_list = ['LAMB', 'AdamW']

# batch_sizes = [8, 32, 64]
# learning_rates = [0.01, 0.001]
# optimizers_list = ['LAMB', 'AdamW']

total_epochs = 250
start=1
step=1

# Load CLIP model and preprocessing
clip_model, _ = clip.load("ViT-B/32", device=device)

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

# Dataset
data_dir = r"../skintypepatches 128x128"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes:", dataset.classes)

# Data Splitting
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


print(f"train size {train_size}")
print(f"Val size {val_size}")
print(f"Test Size {test_size}")

def get_optimizer(optimizer_name, model_params, lr,weight_decay=0.001):
    if optimizer_name == 'AdamW':
        return optim.AdamW(model_params, lr=lr,weight_decay=weight_decay)
    elif optimizer_name=='LAMB':
        return Lamb(model_params, lr=lr,weight_decay=weight_decay)
    
    
# Freeze CLIP vision encoder
for param in clip_model.visual.parameters():
    param.requires_grad = False

# Model definition
class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits

model = CLIPSkinClassifier(clip_model, num_classes=3).to(device).float()



# Metric helper
def compute_metrics(outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    try:
        roc_auc = roc_auc_score(np.eye(3)[labels], F.softmax(outputs, dim=1).cpu().detach().numpy(), multi_class='ovr')
    except:
        roc_auc = None
    rmse = root_mean_squared_error(labels, preds)
    return acc, precision, recall, f1, roc_auc, rmse

# Train loop
def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_outputs = []
    all_labels = []

    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_outputs.append(outputs.detach())
        all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Eval loop
def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device).float(), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_outputs.append(outputs)
            all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Training loop with logging



for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
            
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
            
            # Loss and optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = Lamb(model.parameters(), lr=lr)
            
            for epoch in range(start,total_epochs,step):
                train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
                val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)
                test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)

                print("----------Values After Training-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
                    f"\nOptimizer: {optimizer_name} \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                        f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
                        f"Train ROC AUC: {train_roc_auc:.4f}, Train RMSE: {train_rmse:.4f}")

                print("\n\n-----------Values After Validation-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                            f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc:.4f}, "
                            f"Val RMSE: {val_rmse:.4f}")

                print("\n\n-----------Values After Testing-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                            f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}, Test RMSE: {test_rmse:.4f}")

                # Save results to CSV
                overall_result = {
                    'Epoch': epoch + 1,
                    'Batch Size': batch_size,
                    'Learning Rate': lr,
                    'Optimizer': optimizer_name,

                    'Train Loss': round(train_loss, 4),
                    'Test Loss': round(test_loss, 4),
                    'Val Loss': round(val_loss, 4),

                    'Train Acc': round(train_acc, 4),
                    'Test Acc': round(test_acc, 4),
                    'Val Acc': round(val_acc, 4),

                    'Train Precision': round(train_precision, 4),
                    'Test Precision': round(test_precision, 4),
                    'Val Precision': round(val_precision, 4),

                    'Train Recall': round(train_recall, 4),
                    'Test Recall': round(test_recall, 4),
                    'Val Recall': round(val_recall, 4),

                    'Train F1 Score': round(train_f1, 4),
                    'Test F1 Score': round(test_f1, 4),
                    'Val F1 Score': round(val_f1, 4),

                    'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
                    'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
                    'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,

                    'Train RMSE': round(train_rmse, 4),
                    'Test RMSE': round(test_rmse, 4),
                    'Val RMSE': round(val_rmse, 4)
                }

                overall_result_file = 'sample result (250).csv'
                if not os.path.isfile(overall_result_file):
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
                else:
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)

            # Save model
            os.makedirs('../saved_models', exist_ok=True)
            model_save_path = f"../saved_models/vit_model_bs{batch_size}_lr{lr}_optimizer{optimizer_name}.pth"
            torch.save(model.state_dict(), model_save_path)
            print(f"Model saved to {model_save_path}")




New Model With Clipping (ahmed)

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from torch_optimizer import Lamb
import clip

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Hyperparameters
batch_sizes = [8]
learning_rates = [0.01, 0.001]
optimizers_list = ['LAMB', 'AdamW']
total_epochs = 500
start=100
step=20

# Load CLIP model and preprocessing
clip_model, _ = clip.load("ViT-B/32", device=device)

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

# Dataset
data_dir = r"../skintypepatches 128x128"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes:", dataset.classes)

# Data Splitting
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print(f"train size {train_size}")
print(f"Val size {val_size}")
print(f"Test Size {test_size}")


def get_optimizer(optimizer_name, model_params, lr,weight_decay=0.001):
    if optimizer_name == 'AdamW':
        return optim.AdamW(model_params, lr=lr,weight_decay=weight_decay)
    elif optimizer_name=='LAMB':
        return Lamb(model_params, lr=lr,weight_decay=weight_decay)
    
    
# Freeze CLIP vision encoder
for param in clip_model.visual.parameters():
    param.requires_grad = False

# Model definition
class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits

model = CLIPSkinClassifier(clip_model, num_classes=3).to(device).float()



# Metric helper
def compute_metrics(outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    try:
        roc_auc = roc_auc_score(np.eye(3)[labels], F.softmax(outputs, dim=1).cpu().detach().numpy(), multi_class='ovr')
    except:
        roc_auc = None
    rmse = root_mean_squared_error(labels, preds)
    return acc, precision, recall, f1, roc_auc, rmse

# Train loop
def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_outputs = []
    all_labels = []

    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_outputs.append(outputs.detach())
        all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Eval loop
def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device).float(), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_outputs.append(outputs)
            all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Training loop with logging
for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
            
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
            
            # Loss and optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = Lamb(model.parameters(), lr=lr)
            
            for epoch in range(start,total_epochs,step):
                train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
                val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)
                test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)

                print("----------Values After Training-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
                    f"\nOptimizer: {optimizer_name} \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                        f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
                        f"Train ROC AUC: {train_roc_auc:.4f}, Train RMSE: {train_rmse:.4f}")

                print("\n\n-----------Values After Validation-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                            f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc:.4f}, "
                            f"Val RMSE: {val_rmse:.4f}")

                print("\n\n-----------Values After Testing-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                            f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}, Test RMSE: {test_rmse:.4f}")

                # Save results to CSV
                overall_result = {
                    'Epoch': epoch + 1,
                    'Batch Size': batch_size,
                    'Learning Rate': lr,
                    'Optimizer': optimizer_name,

                    'Train Loss': round(train_loss, 4),
                    'Test Loss': round(test_loss, 4),
                    'Val Loss': round(val_loss, 4),

                    'Train Acc': round(train_acc, 4),
                    'Test Acc': round(test_acc, 4),
                    'Val Acc': round(val_acc, 4),

                    'Train Precision': round(train_precision, 4),
                    'Test Precision': round(test_precision, 4),
                    'Val Precision': round(val_precision, 4),

                    'Train Recall': round(train_recall, 4),
                    'Test Recall': round(test_recall, 4),
                    'Val Recall': round(val_recall, 4),

                    'Train F1 Score': round(train_f1, 4),
                    'Test F1 Score': round(test_f1, 4),
                    'Val F1 Score': round(val_f1, 4),

                    'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
                    'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
                    'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,

                    'Train RMSE': round(train_rmse, 4),
                    'Test RMSE': round(test_rmse, 4),
                    'Val RMSE': round(val_rmse, 4)
                }

                overall_result_file = 'sample result (250).csv'
                if not os.path.isfile(overall_result_file):
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
                else:
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)

            # Save model
            model_dir = '../saved_models(pth)'
            os.makedirs('../saved_models', exist_ok=True)
            model_save_path = f"../saved_models/vit_model_bs{batch_size}_lr{lr}_optimizer{optimizer_name}.pth"
            torch.save(model.state_dict(), model_save_path)
            print(f"Model saved to {model_save_path}")



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Load the dataset
df_results = pd.read_csv('reduced_200_pretrained_overall_result.csv')

# Reshape the dataframe to long format for seaborn
df_long = pd.melt(df_results, id_vars=['Epoch'], var_name='Metric', value_name='Value')

# Optional: Drop rows with NaN values in the Value column (if any)
df_long = df_long.dropna(subset=['Value'])

# Set up the plots with a style
sns.set_theme(style="whitegrid")

# Plot Loss (Train, Test, Val)
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_long[df_long['Metric'].isin(['Train Loss', 'Test Loss', 'Val Loss'])],
             x='Epoch', y='Value', hue='Metric')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(title='Metric')
plt.show()

# Plot Accuracy (Train, Test, Val)
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_long[df_long['Metric'].isin(['Train Acc', 'Test Acc', 'Val Acc'])],
             x='Epoch', y='Value', hue='Metric')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(title='Metric')
plt.show()

# Plot Precision (Train, Test, Val)
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_long[df_long['Metric'].isin(['Train Precision', 'Test Precision', 'Val Precision'])],
             x='Epoch', y='Value', hue='Metric')
plt.title('Precision Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend(title='Metric')
plt.show()

# Plot Recall (Train, Test, Val)
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_long[df_long['Metric'].isin(['Train Recall', 'Test Recall', 'Val Recall'])],
             x='Epoch', y='Value', hue='Metric')
plt.title('Recall Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend(title='Metric')
plt.show()

# Plot F1 Score (Train, Test, Val)
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_long[df_long['Metric'].isin(['Train F1 Score', 'Test F1 Score', 'Val F1 Score'])],
             x='Epoch', y='Value', hue='Metric')
plt.title('F1 Score Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend(title='Metric')
plt.show()

# Plot ROC AUC (Train, Test, Val)
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_long[df_long['Metric'].isin(['Train ROC AUC', 'Test ROC AUC', 'Val ROC AUC'])],
             x='Epoch', y='Value', hue='Metric')
plt.title('ROC AUC Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('ROC AUC')
plt.legend(title='Metric')
plt.show()

# Plot RMSE (Train, Test, Val)
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_long[df_long['Metric'].isin(['Train RMSE', 'Test RMSE', 'Val RMSE'])],
             x='Epoch', y='Value', hue='Metric')
plt.title('RMSE Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.legend(title='Metric')
plt.show()


Akhati report (12 vlaues in each graph)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from docx import Document
from docx.shared import Inches
import os

# Load CSV
df = pd.read_csv('reduced_200_pretrained_overall_result.csv')

# Set correct column names
lr_col = 'Learning Rate'
bs_col = 'Batch Size'
opt_col = 'Optimizer'
epoch_col = 'Epoch'

# Metrics to plot
metrics = [ 
    'Train Loss', 'Test Loss', 'Val Loss',
    'Train Acc', 'Test Acc', 'Val Acc',
    'Train Precision', 'Test Precision', 'Val Precision',
    'Train Recall', 'Test Recall', 'Val Recall',
    'Train F1 Score', 'Test F1 Score', 'Val F1 Score',
    'Train ROC AUC', 'Test ROC AUC', 'Val ROC AUC',
    'Train RMSE', 'Test RMSE', 'Val RMSE'
]

# Create unique configuration name
df['config'] = df[lr_col].astype(str) + '_lr_' + \
               df[bs_col].astype(str) + '_bs_' + \
               df[opt_col]

# Set plot style
sns.set(style='whitegrid')

# Create a directory to save images
image_dir = "metric_plots"
os.makedirs(image_dir, exist_ok=True)

# Create Word document
doc = Document()
doc.add_heading('Training Metrics Report', 0)

# Generate plots and add to document
for metric in metrics:
    plt.figure(figsize=(10, 6))
    for config_name, group in df.groupby('config'):
        plt.plot(group[epoch_col], group[metric], label=config_name)

    plt.title(metric)
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend(title='Config', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    
    # Save plot as image
    image_path = os.path.join(image_dir, f"{metric.replace(' ', '_')}.png")
    plt.savefig(image_path)
    plt.close()

    # Add title and image to Word doc
    doc.add_heading(metric, level=1)
    doc.add_picture(image_path, width=Inches(6.5))  # Adjust width if needed

# Save the Word document
doc.save('Reduced 200 Images Training Metrics Report.docx')
print("Word document 'Training_Metrics_Report.docx' created with all plots.")


Alag alag report based on (learning rate, batch size, optimizer )

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from docx import Document
from docx.shared import Inches
import os

# Load the data
df = pd.read_csv("reduced_200_pretrained_overall_result.csv")

# Combine hyperparameters into a config string for reference (not used in graph)
df['Config'] = df.apply(lambda row: f"LR={row['Learning Rate']} | BS={row['Batch Size']} | Opt={row['Optimizer']}", axis=1)

# Set Seaborn style
sns.set_theme(style="whitegrid")

# List of metrics to visualize
metrics = [
    'Train Loss', 'Test Loss', 'Val Loss',
    'Train Acc', 'Test Acc', 'Val Acc',
    'Train Precision', 'Test Precision', 'Val Precision',
    'Train Recall', 'Test Recall', 'Val Recall',
    'Train F1 Score', 'Test F1 Score', 'Val F1 Score',
    'Train ROC AUC', 'Test ROC AUC', 'Val ROC AUC',
    'Train RMSE', 'Test RMSE', 'Val RMSE'
]

# Create a folder for images
img_folder = "metric_images"
os.makedirs(img_folder, exist_ok=True)

# Create Word document
doc = Document()
doc.add_heading("Model Evaluation Metrics Report", 0)

# Loop over each metric, create and save plot, insert into doc
for metric in metrics:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Plot by Learning Rate
    sns.lineplot(data=df, x='Epoch', y=metric, hue='Learning Rate', marker='o', ax=axes[0])
    axes[0].set_title(f"{metric} vs Epoch (Learning Rate)")
    axes[0].legend(title="LR", fontsize=7)

    # Plot by Batch Size
    sns.lineplot(data=df, x='Epoch', y=metric, hue='Batch Size', marker='o', ax=axes[1])
    axes[1].set_title(f"{metric} vs Epoch (Batch Size)")
    axes[1].legend(title="BS", fontsize=7)

    # Plot by Optimizer
    sns.lineplot(data=df, x='Epoch', y=metric, hue='Optimizer', marker='o', ax=axes[2])
    axes[2].set_title(f"{metric} vs Epoch (Optimizer)")
    axes[2].legend(title="Opt", fontsize=7)

    # Save figure
    image_path = os.path.join(img_folder, f"{metric.replace(' ', '_')}.png")
    plt.tight_layout()
    fig.savefig(image_path, dpi=300)
    plt.close(fig)

    # Add to Word doc
    doc.add_heading(metric, level=1)
    doc.add_picture(image_path, width=Inches(6.5))  # Use full width, fits well

# Save final document
doc.save("Reduced 200 Images Metric Evaluation Report.docx")
print("✅ All metric graphs saved and Word document generated as 'Metric_Evaluation_Report.docx'.")


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from torch_optimizer import Lamb
import clip

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Hyperparameters
batch_sizes = [8]
learning_rates = [0.001]
optimizers_list = ['LAMB', 'AdamW']
total_epochs = 500
start=100
step=20

# Load CLIP model and preprocessing
clip_model, _ = clip.load("ViT-B/32", device=device)

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

# Dataset
data_dir = r"../skintypepatches 128x128"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes:", dataset.classes)

# Data Splitting
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print(f"train size {train_size}")
print(f"Val size {val_size}")
print(f"Test Size {test_size}")


def get_optimizer(optimizer_name, model_params, lr,weight_decay=0.001):
    if optimizer_name == 'AdamW':
        return optim.AdamW(model_params, lr=lr,weight_decay=weight_decay)
    elif optimizer_name=='LAMB':
        return Lamb(model_params, lr=lr,weight_decay=weight_decay)
    
    
# Freeze CLIP vision encoder
for param in clip_model.visual.parameters():
    param.requires_grad = False

# Model definition
class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits

model = CLIPSkinClassifier(clip_model, num_classes=3).to(device).float()



# Metric helper
def compute_metrics(outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    try:
        roc_auc = roc_auc_score(np.eye(3)[labels], F.softmax(outputs, dim=1).cpu().detach().numpy(), multi_class='ovr')
    except:
        roc_auc = None
    rmse = root_mean_squared_error(labels, preds)
    return acc, precision, recall, f1, roc_auc, rmse

# Train loop
def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_outputs = []
    all_labels = []

    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_outputs.append(outputs.detach())
        all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Eval loop
def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device).float(), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_outputs.append(outputs)
            all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Training loop with logging
for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
            
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
            
            # Loss and optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = Lamb(model.parameters(), lr=lr)
            
            for epoch in range(start,total_epochs,step):
                train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
                val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)
                test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)

                print("----------Values After Training-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
                    f"\nOptimizer: {optimizer_name} \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                        f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
                        f"Train ROC AUC: {train_roc_auc:.4f}, Train RMSE: {train_rmse:.4f}")

                print("\n\n-----------Values After Validation-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                            f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc:.4f}, "
                            f"Val RMSE: {val_rmse:.4f}")

                print("\n\n-----------Values After Testing-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                            f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}, Test RMSE: {test_rmse:.4f}")

                # Save results to CSV
                overall_result = {
                    'Epoch': epoch + 1,
                    'Batch Size': batch_size,
                    'Learning Rate': lr,
                    'Optimizer': optimizer_name,

                    'Train Loss': round(train_loss, 4),
                    'Test Loss': round(test_loss, 4),
                    'Val Loss': round(val_loss, 4),

                    'Train Acc': round(train_acc, 4),
                    'Test Acc': round(test_acc, 4),
                    'Val Acc': round(val_acc, 4),

                    'Train Precision': round(train_precision, 4),
                    'Test Precision': round(test_precision, 4),
                    'Val Precision': round(val_precision, 4),

                    'Train Recall': round(train_recall, 4),
                    'Test Recall': round(test_recall, 4),
                    'Val Recall': round(val_recall, 4),

                    'Train F1 Score': round(train_f1, 4),
                    'Test F1 Score': round(test_f1, 4),
                    'Val F1 Score': round(val_f1, 4),

                    'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
                    'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
                    'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,

                    'Train RMSE': round(train_rmse, 4),
                    'Test RMSE': round(test_rmse, 4),
                    'Val RMSE': round(val_rmse, 4)
                }

                overall_result_file = 'sample result (250).csv'
                if not os.path.isfile(overall_result_file):
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
                else:
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)

            # Save model
            os.makedirs('../saved_models', exist_ok=True)
            model_save_path = f"../saved_models/vit_model_bs{batch_size}_lr{lr}_optimizer{optimizer_name}.pth"
            torch.save(model.state_dict(), model_save_path)
            print(f"Model saved to {model_save_path}")



In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from torch_optimizer import Lamb  # You may need to install this via pip install pytorch-optimizer
import clip
from torch.optim import AdamW

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Hyperparameters
batch_size = 8
lr = 0.0001
optimizer_name = "AdamW"
total_epochs = 501
start=99
step=20

# Load CLIP model and preprocessing
clip_model, _ = clip.load("ViT-B/32", device=device)

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

# Dataset
data_dir = "../skintypepatches 128x128"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes:", dataset.classes)

# Data Splitting
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Freeze CLIP vision encoder
for param in clip_model.visual.parameters():
    param.requires_grad = False

# Model definition
class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits

model = CLIPSkinClassifier(clip_model, num_classes=3).to(device).float()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = Lamb(model.parameters(), lr=lr)
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=0.001)
# Metric helper
def compute_metrics(outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    try:
        roc_auc = roc_auc_score(np.eye(3)[labels], F.softmax(outputs, dim=1).cpu().detach().numpy(), multi_class='ovr')
    except:
        roc_auc = None
    rmse = root_mean_squared_error(labels, preds)
    return acc, precision, recall, f1, roc_auc, rmse

# Train loop
def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_outputs = []
    all_labels = []

    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_outputs.append(outputs.detach())
        all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Eval loop
def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device).float(), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_outputs.append(outputs)
            all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Training loop with logging
for epoch in range(start,total_epochs,step):
    train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)
    test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)

    print("----------Values After Training-----------")
    print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
          f"\nOptimizer: {optimizer_name} \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
               f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
               f"Train ROC AUC: {train_roc_auc:.4f}, Train RMSE: {train_rmse:.4f}")

    print("\n\n-----------Values After Validation-----------")
    print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
          f"\nOptimizer: {optimizer_name} \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc:.4f}, "
                f"Val RMSE: {val_rmse:.4f}")

    print("\n\n-----------Values After Testing-----------")
    print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
          f"\nOptimizer: {optimizer_name} Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}, Test RMSE: {test_rmse:.4f}")

    # Save results to CSV
    overall_result = {
        'Epoch': epoch + 1,
        'Batch Size': batch_size,
        'Learning Rate': lr,
        'Optimizer': optimizer_name,

        'Train Loss': round(train_loss, 4),
        'Test Loss': round(test_loss, 4),
        'Val Loss': round(val_loss, 4),

        'Train Acc': round(train_acc, 4),
        'Test Acc': round(test_acc, 4),
        'Val Acc': round(val_acc, 4),

        'Train Precision': round(train_precision, 4),
        'Test Precision': round(test_precision, 4),
        'Val Precision': round(val_precision, 4),

        'Train Recall': round(train_recall, 4),
        'Test Recall': round(test_recall, 4),
        'Val Recall': round(val_recall, 4),

        'Train F1 Score': round(train_f1, 4),
        'Test F1 Score': round(test_f1, 4),
        'Val F1 Score': round(val_f1, 4),

        'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
        'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
        'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,

        'Train RMSE': round(train_rmse, 4),
        'Test RMSE': round(test_rmse, 4),
        'Val RMSE': round(val_rmse, 4)
    }

    overall_result_file = 'test 4.csv'
    if not os.path.isfile(overall_result_file):
        pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
    else:
        pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)

Using device: cuda
Classes: ['dry', 'normal', 'oily']


100%|██████████| 346/346 [00:20<00:00, 16.78it/s]


----------Values After Training-----------

Epoch: [100/501] 
Batch Size: 8 
Learning Rate: 0.0001 
Optimizer: AdamW 
Train Loss: 1.0792, Train Acc: 0.4043, Train Precision: 0.4042, Train Recall: 0.4044, Train  F1: 0.4041,Train ROC AUC: 0.5831, Train RMSE: 1.0946


-----------Values After Validation-----------

Epoch: [100/501] 
Batch Size: 8 
Learning Rate: 0.0001
Optimizer: AdamW 
Val Loss: 1.0687, Val Acc: 0.4259, Val Precision: 0.4245, Val Recall: 0.4235, Val F1: 0.4162, Val ROC AUC: 0.6077, Val RMSE: 1.0389


-----------Values After Testing-----------

Epoch: [100/501] 
Batch Size: 8 
Learning Rate: 0.0001
Optimizer: AdamW Test Loss: 1.0621, Test Acc: 0.4222, Test Precision: 0.4252, Test Recall: 0.4251, Test F1: 0.4164, Test ROC AUC: 0.6204, Test RMSE: 1.0406


100%|██████████| 346/346 [00:18<00:00, 18.42it/s]


----------Values After Training-----------

Epoch: [120/501] 
Batch Size: 8 
Learning Rate: 0.0001 
Optimizer: AdamW 
Train Loss: 1.0555, Train Acc: 0.4443, Train Precision: 0.4442, Train Recall: 0.4443, Train  F1: 0.4441,Train ROC AUC: 0.6246, Train RMSE: 1.0615


-----------Values After Validation-----------

Epoch: [120/501] 
Batch Size: 8 
Learning Rate: 0.0001
Optimizer: AdamW 
Val Loss: 1.0601, Val Acc: 0.4353, Val Precision: 0.4473, Val Recall: 0.4346, Val F1: 0.4235, Val ROC AUC: 0.6240, Val RMSE: 1.1206


-----------Values After Testing-----------

Epoch: [120/501] 
Batch Size: 8 
Learning Rate: 0.0001
Optimizer: AdamW Test Loss: 1.0499, Test Acc: 0.4526, Test Precision: 0.4637, Test Recall: 0.4529, Test F1: 0.4407, Test ROC AUC: 0.6376, Test RMSE: 1.0980


...

----------Values After Training-----------

Epoch: [460/501] 
Batch Size: 8 
Learning Rate: 0.0001 
Optimizer: AdamW 
Train Loss: 0.8362, Train Acc: 0.6249, Train Precision: 0.6249, Train Recall: 0.6249, Train  F1: 0.6248,Train ROC AUC: 0.8075, Train RMSE: 0.8573


-----------Values After Validation-----------

Epoch: [460/501] 
Batch Size: 8 
Learning Rate: 0.0001
Optimizer: AdamW 
Val Loss: 0.9808, Val Acc: 0.5217, Val Precision: 0.5309, Val Recall: 0.5241, Val F1: 0.5197, Val ROC AUC: 0.7126, Val RMSE: 0.9862


-----------Values After Testing-----------

Epoch: [460/501] 
Batch Size: 8 
Learning Rate: 0.0001
Optimizer: AdamW Test Loss: 0.9552, Test Acc: 0.5398, Test Precision: 0.5478, Test Recall: 0.5372, Test F1: 0.5369, Test ROC AUC: 0.7251, Test RMSE: 0.9775


100%|██████████| 346/346 [00:10<00:00, 34.60it/s]


----------Values After Training-----------

Epoch: [480/501] 
Batch Size: 8 
Learning Rate: 0.0001 
Optimizer: AdamW 
Train Loss: 0.8236, Train Acc: 0.6337, Train Precision: 0.6337, Train Recall: 0.6337, Train  F1: 0.6337,Train ROC AUC: 0.8145, Train RMSE: 0.8493


-----------Values After Validation-----------

Epoch: [480/501] 
Batch Size: 8 
Learning Rate: 0.0001
Optimizer: AdamW 
Val Loss: 0.9787, Val Acc: 0.5275, Val Precision: 0.5358, Val Recall: 0.5298, Val F1: 0.5257, Val ROC AUC: 0.7166, Val RMSE: 0.9791


-----------Values After Testing-----------

Epoch: [480/501] 
Batch Size: 8 
Learning Rate: 0.0001
Optimizer: AdamW Test Loss: 0.9490, Test Acc: 0.5456, Test Precision: 0.5509, Test Recall: 0.5429, Test F1: 0.5422, Test ROC AUC: 0.7311, Test RMSE: 0.9588


100%|██████████| 346/346 [00:09<00:00, 34.66it/s]

----------Values After Training-----------

Epoch: [500/501] 
Batch Size: 8 
Learning Rate: 0.0001 
Optimizer: AdamW 
Train Loss: 0.8071, Train Acc: 0.6452, Train Precision: 0.6452, Train Recall: 0.6452, Train  F1: 0.6452,Train ROC AUC: 0.8245, Train RMSE: 0.8314


-----------Values After Validation-----------

Epoch: [500/501] 
Batch Size: 8 
Learning Rate: 0.0001
Optimizer: AdamW 
Val Loss: 0.9669, Val Acc: 0.5311, Val Precision: 0.5336, Val Recall: 0.5322, Val F1: 0.5310, Val ROC AUC: 0.7204, Val RMSE: 0.9588


-----------Values After Testing-----------

Epoch: [500/501] 
Batch Size: 8 
Learning Rate: 0.0001
Optimizer: AdamW Test Loss: 0.9461, Test Acc: 0.5380, Test Precision: 0.5389, Test Recall: 0.5367, Test F1: 0.5369, Test ROC AUC: 0.7315, Test RMSE: 0.9497
